In [ ]:
#p301 시군구별 인구 단계 구분도
import json
import pandas as pd
geo = json.load(open('SIG.geojson', encoding = 'UTF-8'))
#행정 구역 코드 출력
geo['features'][0]['properties']
#위도, 경도 좌표 출력
geo['features'][0]['geometry']

df_pop = pd.read_csv('Population_SIG.csv')
df_pop.head()
df_pop.info()
#행정 구역 코드 문자 타입으로 변경
df_pop['code'] = df_pop['code'].astype(str)

import folium
folium.Map(location = [35.95, 127.7], #지도 중심 좌표
           zoom_start = 8) #확대 단계
#계급 구간 나누기
bins = list(df_pop['pop'].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))
#배경 지도 만들기
map_sig = folium.Map(location = [35.95, 127.7],
                     zoom_start = 8,
                     tiles = 'cartodbpositron') #지도 종류
#단계 구분도 만들기
folium.Choropleth(
    geo_data = geo, #지도 데이터
    data = df_pop, #색깔로 표현할 통계 데이터
    columns = ('code', 'pop'),
    key_on = 'feature.properties.SIG_CD', #geo 행정 구역 코드
    fill_color = 'YlGnBu', #컬러맵
    fill_opacity = 1, #투명도
    line_opacity = 0.5, #경계선 투명도
    bins = bins)\
        .add_to(map_sig)
map_sig

In [ ]:
#p308 서울시 동별 외국인 인구 단계 구분도
import json
geo_seoul = json.load(open('EMD_Seoul.geojson', encoding = 'UTF-8'))
#행정 구역 코드 출력
geo_seoul['features'][0]['properties']
#위도, 경도 좌표 출력
geo_seoul['features'][0]['geometry']

foreigner = pd.read_csv('Foreigner_EMD_Seoul.csv')
foreigner.head()
foreigner.info()

#foreigner code 문자 타입으로 변경
foreigner['code'] = foreigner['code'].astype(str)
bins = list(foreigner['pop'].quantile([0, 0.2, 0.4, 0.5, 0.6, 
                                       0.7, 0.8, 0.9, 1]))
#배경 지도
map_seoul = folium.Map(location = [37.56, 127], #서울 좌표
                       zoom_start = 12,
                       tiles = 'cartodbpositron')
#단계구분도
folium.Choropleth(
    geo_data = geo_seoul,
    data = foreigner,
    columns = ('code', 'pop'),
    key_on = 'feature.properties.ADM_DR_CD',
    fill_color = 'Blues',
    nan_fill_color = 'White',
    fill_opacity = 1,
    line_opacity = 0.5,
    bins = bins)\
        .add_to(map_seoul)

#구 경계선 추가
geo_seoul_sig = json.load(open('SIG_Seoul.geojson', encoding = 'UTF-8'))
folium.Choropleth(geo_data = geo_seoul_sig,
                  fill_opacity = 0,
                  line_weight = 4)\
      .add_to(map_seoul)
map_seoul

#html파일로 저장
map_seoul.save('map_seoul.html')
#웹브라우저에서 open
import webbrowser
webbrowser.open_new('map_seoul.html')

In [ ]:
#인터랙티브 그래프
#p316 산점도
import pandas as pd
import plotly.express as px
mpg = pd.read_csv('mpg.csv')

#산점도 만들기
px.scatter(data_frame = mpg, x = 'cty', y = 'hwy', color = 'drv') #파라미터 data_frame 주의
#마우스 드래그 -> x, y축 범위 변경
#그래프 더블클릭 -> 원래대로 돌아옴

In [ ]:
#p318 막대 그래프
import pandas as pd
import plotly.express as px
mpg = pd.read_csv('mpg.csv')
df = mpg.groupby('category', as_index = False)\
        .agg(n = ('category', 'count'))

#막대 그래프 만들기
px.bar(data_frame = df, x = 'category', y = 'n', color = 'category')

In [ ]:
#p319 선 그래프
import pandas as pd
import plotly.express as px
economics = pd.read_csv('economics.csv')

#선 그래프 만들기
px.line(data_frame = economics, x = 'date', y = 'psavert')

In [ ]:
#p320 상자 그림
import pandas as pd
import plotly.express as px
mpg = pd.read_csv('mpg.csv')

#상자 그림 만들기
px.box(data_frame = mpg, x = 'drv', y = 'hwy', color = 'drv')

#그래프 변수에 할당
fig = px.scatter(data_frame = mpg, x = 'cty', y = 'hwy')
#html로 저장
#fig.write_html('scatter_plot.html')
#크기조정은 width, height
#웹브라우저 창에 그래프 출력
#plotly.io.renderers.default = 'browser'
#설정 원래대로 되돌림
#plotly.io.renderers.default = 'jupyterlab'

In [ ]:
#t 검정(두 집단 평균 통계적으로 유의한 차 알아보기)
import pandas as pd
mpg = pd.read_csv('mpg.csv')

#compact 자동차와 suv 자동차 도시 연비 t 검정
#기술 통계 분석
mpg.query('category in ["compact", "suv"]')\
   .groupby('category', as_index = False)\
   .agg(n = ('category', 'count'),
        mean = ('cty', 'mean'))
compact = mpg.query('category == "compact"')['cty']
suv = mpg.query('category == "suv"')['cty']

#t-test
from scipy import stats
stats.ttest_ind(compact, suv, equal_var = True)
#pvalue가 0.05 미만이면 집단 간 차이 통계적으로 유의미함

#일반 휘발유와 고급 휘발유의 도시 연비 t 검정
#기술 통계 분석
mpg.query('fl in ["r", "p"]')\
   .groupby('fl', as_index = False)\
   .agg(n = ('fl', 'count'),
        mean = ('cty', 'mean'))
regular = mpg.query('fl == "r"')['cty']
premium = mpg.query('fl == "p"')['cty']

#t-test
stats.ttest_ind(regular, premium, equal_var = True)
#pvalue가 0.05보다 큰 0.2875...

In [ ]:
#상관분석(두 변수 관계 분석)
import pandas as pd
economics = pd.read_csv('economics.csv')

#상관행렬 만들기
economics[['unemploy', 'pce']].corr() #상관계수 양수 0.61

#유의확률
from scipy import stats
stats.pearsonr(economics['unemploy'], economics['pce']) #첫 값이 상관계수, 두 번째가 유의확률


mtcars = pd.read_csv('mtcars.csv')
mtcars.head()
#상관행렬 만들기
car_cor = mtcars.corr()
car_cor = round(car_cor, 2) #소수점 둘째 자리까지 반올림
car_cor

import matplotlib.pyplot as plt
plt.rcParams.update({'figure.dpi': '120', #해상도 설정
                     'figure.figsize': [7.5, 5.5]}) #가로 세로 크기 설정
#히트맵 만들기
import seaborn as sns
#sns.heatmap(car_cor,
            #annot = True, #상관계수 표시
            #cmap = 'RdBu') #컬러맵

#대각 행렬 제거
#mask 만들기
import numpy as np
mask = np.zeros_like(car_cor)
#오른쪽 위 대각 행렬을 1로 바꾸기
mask[np.triu_indices_from(mask)] = 1
#히트맵에 mask 적용
#sns.heatmap(data = car_cor,
            #annot = True, #상관계수 표시
            #cmap = 'RdBu',#컬러맵
            #mask = mask)

#빈 행과 열 제거
mask_new = mask[1:, :-1] #mask 첫 번째 행과 마지막 열 제거
cor_new = car_cor.iloc[1:, :-1] #상관행렬 첫 번째 행과 마지막 열 제거
#히트맵에 mask 적용
#sns.heatmap(data = cor_new,
            #annot = True, #상관계수 표시
            #cmap = 'RdBu',#컬러맵
            #mask = mask_new)

#파라미터 추가 -> 보기 좋게 수정
sns.heatmap(data = cor_new,
            annot = True, #상관계수 표시
            cmap = 'RdBu',#컬러맵
            mask = mask_new,
            linewidths = .5, #경계 구분선 추가
            vmax = 1, #가장 진한 파란색으로 표현할 최대값
            vmin = -1, #가장 진한 빨간색으로 표현할 최소값
            cbar_kws = {'shrink': .5}) #범례 크기 줄이기